# finda_media_sa_prep_20220926

### 디렉토리 설정
- 아래 각 변수에 해당하는 알맞은 경로를 찾아 입력해주세요.
- 경로 주소는 '.../.../...' 형식을 따라주세요.

In [ ]:
class directory:
    dropbox_dir = 'C:/Users/MADUP/Dropbox (주식회사매드업)'
    # 드롭박스 폴더 위치경로
    
    download_dir = dropbox_dir  + '/광고사업부/4. 광고주/핀다_7팀/2. 리포트/데일리리포트_오토&SA/SA 앱스플라이어 가공/SA 매체자동화리포트'
    # 최종 결과파일을 다운받을 폴더 위치경로

dr = directory()

### 날짜 설정
- 보통 작일자 기준으로 요청주신 기간의 데이터를 출력하게 됩니다.
- 날짜 조정이 필요한 경우 아래 변수 값을 수정하여 출력해주세요.

In [ ]:
import datetime

class report_date:
    today = datetime.date.today()
    # 오늘 날짜 ex> 2022-09-14
    
    # today = datetime.date(year = 2022, month = 8, day = 12)
    # 다른 일자를 기준으로 추출하고 싶은 경우 위 코드 수정 및 주석해제
    
    day_1 = today - datetime.timedelta(1)
    # 전일자 ex> 2022-09-13
    
    start_day = datetime.date(year=day_1.year, month=day_1.month, day=1)
    # 금월 첫 1일자 ex> 2022-09-01
    
    yearmonth = day_1.strftime('%Y%m')
    # 년월 ex> 202209
    
    month_name = str(day_1.month) + '월'
    # 금월 ex> 9월

rdate = report_date()

### 구글 스프레드 시트 읽기
- 특정 구글 스프레드 시트를 읽어오기 위해 활용하는 메소드 모음입니다.
- dcteam@madup-355605.iam.gserviceaccount.com 계정에 참조하고자 하는 구글 시트 공유가 필요합니다.

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import time


class spreadsheet:    
    def spread_document_read(self, url):
        token_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/token'
        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        json_file = token_dir + '/madup-355605-cd37b0ac201f.json'
        credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file, scope)
        gc = gspread.authorize(credentials)
        read = gc.open_by_url(url)
        print('스프레드시트 읽기 완료')
        return read
    
    def spread_sheet(self, doc, sheet_name, col_num=0, row_num=0):
        while True :
            try :
                data_sheet = doc.worksheet(sheet_name)
                data_sheet_read = data_sheet.get_all_values()
                result = pd.DataFrame(data_sheet_read, columns=data_sheet_read[row_num]).iloc[row_num+1:, col_num:]
            except :
                print('API 오류로 15초 후 다시 시도 합니다.')
                time.sleep(15)
                continue
            break
        return result

spreadsheet = spreadsheet()

### 코드
- 전처리 자동화를 위한 코드로 임의 수정을 하실 경우 복사본 생성하여 진행해주시길 바랍니다.
- 혹 아래 이미지와 같은 형식의 오류가 발생할 경우 `!pip install 패키지명`형식의 코드 작성하여 실행해주시면 해당 패키지가 설치됩니다.
    - <img src = 'https://ifh.cc/g/ymACFT.png' align='left'>

- 이외의 오류가 발생하거나 문제가 해결되지 않는 경우, 데컨팀에 문의주세요 :)

In [ ]:
# 패키지 설치 코드 (패키지 설치 오류 발생시 아래 코드 실행해주세요.)
# !pip install xlsxwriter
# !pip install pyarrow

In [ ]:
import os
import pyarrow as pa
import pyarrow.csv as pacsv
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


finda_raw_dir = dr.dropbox_dir + '/광고사업부/4. 광고주/핀다_7팀/2. 리포트/자동화리포트'
n_raw_dir = finda_raw_dir + '/naver_prism'
g_raw_dir = finda_raw_dir + '/google_sa_prism'
result_dir = dr.download_dir
required_date = rdate.day_1


def get_raw_df(raw_dir, required_date):
    if 'naver' in raw_dir:
        dtypes = {
            'date': pa.string(),
            'campaign_name': pa.string(),
            'adgroup_name': pa.string(),
            'ad_keyword': pa.string(),
            'ad_id': pa.string(),
            'impression': pa.string(),
            'click': pa.string(),
            'cost': pa.string()
        }
    elif 'google' in raw_dir:
        dtypes = {
            'segments_date': pa.string(),
            'campaign_name': pa.string(),
            'ad_group_name': pa.string(),
            'segments_keyword_info_text': pa.string(),
            'ad_group_ad_ad_final_urls': pa.string(),
            'impressions': pa.string(),
            'clicks': pa.string(),
            'cost_micros': pa.string()
        }

    index_columns = list(dtypes.keys())
    convert_ops = pacsv.ConvertOptions(column_types=dtypes, include_columns=index_columns)
    ro = pacsv.ReadOptions(block_size=10 << 20)
    table_list = []

    files = os.listdir(raw_dir)
    files = [f for f in files if '.csv' in f]

    date = required_date.strftime('%Y%m')
    raw_files = [f for f in files if date in str(f)]

    for f in raw_files:
        temp = pacsv.read_csv(raw_dir + '/' + f, convert_options=convert_ops, read_options=ro)
        table_list.append(temp)
    table = pa.concat_tables(table_list)
    raw_df = table.to_pandas()
    return raw_df

def prep_df(n_raw_df, g_raw_df, doc):
    n_raw_df = n_raw_df.loc[~(n_raw_df['campaign_name'] == '핀다_BS')]
    # n_raw_df = n_raw_df.loc[~(n_raw_df['campaign_name'] != '0')]
    n_raw_df['date'] = n_raw_df['date'].apply(lambda x: pd.to_datetime(x).date())
    n_raw_df[['impression','click','cost']] = n_raw_df[['impression','click','cost']].apply(lambda x: pd.to_numeric(x))
    n_raw_df.loc[:, 'cost'] = n_raw_df.loc[:, 'cost'].apply(lambda x: x * 1.1)
    # 네이버 가공

    g_raw_df['segments_date'] = g_raw_df['segments_date'].apply(lambda x: pd.to_datetime(x).date())
    g_raw_df[['impressions', 'clicks', 'cost_micros']] = g_raw_df[['impressions', 'clicks', 'cost_micros']].apply(lambda x: pd.to_numeric(x))
    g_raw_df['cost_micros'] = g_raw_df['cost_micros'].apply(lambda x: x / 1000000)
    g_raw_df['sum_metrix'] = g_raw_df['impressions'] + g_raw_df['clicks'] + g_raw_df['cost_micros']
    g_raw_df = g_raw_df.loc[g_raw_df['sum_metrix'] > 0]
    def utm_content(list):
        for x in list:
            if 'utm_content' in x:
                return x
    g_raw_df['ad_group_ad_ad_final_urls'] = g_raw_df['ad_group_ad_ad_final_urls'].apply(lambda x: utm_content(x.split('&'))[12:])
    sheet_data = spreadsheet.spread_sheet(doc, '구글라벨(자동화:터치NO)') # 소재명-브릿지페이지 매핑
    label_dict = dict(zip(sheet_data['광고 소재명'], sheet_data['브릿지 페이지명']))
    g_raw_df['ad_group_ad_ad_final_urls'] = g_raw_df['ad_group_ad_ad_final_urls'].apply(lambda x: x+label_dict[x])
    # 구글 가공

    return n_raw_df, g_raw_df

def download_df(n_result_df, g_result_df):
    # 결과파일 다운로드
    date = required_date.strftime('%Y%m%d')
    writer = pd.ExcelWriter(result_dir + f'/매체종합_{date}.xlsx', engine='xlsxwriter', engine_kwargs={'options': {'strings_to_urls': False}})
    n_result_df.to_excel(writer, sheet_name='naver', encoding='utf-8-sig', index=False)
    g_result_df.to_excel(writer, sheet_name='google', encoding='utf-8-sig', index=False)
    writer.close()
    print('download success')


n_raw_df = get_raw_df(n_raw_dir, required_date)
g_raw_df = get_raw_df(g_raw_dir, required_date)
doc = spreadsheet.spread_document_read(
    'https://docs.google.com/spreadsheets/d/1DEDtSe3LGC2bqS3-UHL62h50vtLIvzkbk3Nx512zZow/edit#gid=2004266056')
n_result_df, g_result_df = prep_df(n_raw_df, g_raw_df, doc)
download_df(n_result_df, g_result_df)


---